In [1]:
using Rx
using BenchmarkTools

┌ Info: Precompiling Rx [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1273


In [2]:
function withRegularArray()
    values = [ i for i in 1:1000 ]
    keep = Array{Int, 1}()
    for value in values
        if (value % 2 == 0)
            push!(keep, value)
        end
    end
    return keep
end

withRegularArray (generic function with 1 method)

In [3]:
struct KeepActor{D} <: NextActor{D}
    values::Array{D, 1}
    
    KeepActor{D}() where D = new(Array{D, 1}())
end

Rx.on_next!(actor::KeepActor{D}, data::D) where D = push!(actor.values, data)

In [4]:
@CreateFilterOperator(Even, (d) -> d % 2 == 0)

function withPregeneratedOperator()
    source = from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(source |> EvenFilterOperator{Int}(), keep)
    return keep.values
end

withPregeneratedOperator (generic function with 1 method)

In [5]:
function withBaseFilterOperator()
    source = from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(source |> filter(Int, (d) -> d % 2 == 0), keep)
    return keep.values
end

withBaseFilterOperator (generic function with 1 method)

In [6]:
@btime withRegularArray();

  4.069 μs (10 allocations: 16.27 KiB)


In [7]:
@btime withPregeneratedOperator();

  4.202 μs (14 allocations: 16.34 KiB)


In [8]:
@btime withBaseFilterOperator();

  64.226 μs (1749 allocations: 43.47 KiB)
